#  Семинар 3. userkNN  CV для сравнения моделей

- Сравниваем две модели implicit `CosineRecommender` and `TFIDFRecommender` 



In [1]:
import warnings
from pprint import pprint

import pandas as pd
import requests
from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender
from rectools import Columns
from rectools.dataset import Interactions
from rectools.metrics import MAP, MeanInvUserFreq, calc_metrics
from rectools.model_selection import TimeRangeSplitter
from tqdm.auto import tqdm

from userknn import UserKnn

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

# Датасет KION 

In [2]:
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'
req = requests.get(url, stream=True)

with open('kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='Downloading the kion dataset...', 
                        total=total_size_in_bytes, 
                        unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

In [3]:
!unzip kion_train.zip -x '__MACOSX/*'

"unzip" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [5]:
interactions_df = pd.read_csv('../datasets/interactions.csv')
users = pd.read_csv('../datasets/users.csv')
items = pd.read_csv('../datasets/items.csv')

interactions_df.rename(columns={'last_watch_dt': Columns.Datetime,
                                'total_dur': Columns.Weight}, inplace=True) 
# will cast types and save new pd.DataFrame inside in Interactions.df
interactions = Interactions(interactions_df)   

# ! если хотите быстро прогнать этот ноутбук - раскомментируйте эту строку - она уменьшает данные
# interactions = Interactions(interactions_df.sample(frac=0.01))  

interactions.df.head()

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0


## Задаем фолды для кросс-валидации

In [6]:
N_SPLITS = 7
TEST_SIZE = '7D'

In [7]:
# Init generator of folds
cv = TimeRangeSplitter(
    test_size=TEST_SIZE,
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

In [8]:
cv.get_test_fold_borders(interactions)

[(Timestamp('2021-07-05 00:00:00', freq='7D'),
  Timestamp('2021-07-12 00:00:00', freq='7D')),
 (Timestamp('2021-07-12 00:00:00', freq='7D'),
  Timestamp('2021-07-19 00:00:00', freq='7D')),
 (Timestamp('2021-07-19 00:00:00', freq='7D'),
  Timestamp('2021-07-26 00:00:00', freq='7D')),
 (Timestamp('2021-07-26 00:00:00', freq='7D'),
  Timestamp('2021-08-02 00:00:00', freq='7D')),
 (Timestamp('2021-08-02 00:00:00', freq='7D'),
  Timestamp('2021-08-09 00:00:00', freq='7D')),
 (Timestamp('2021-08-09 00:00:00', freq='7D'),
  Timestamp('2021-08-16 00:00:00', freq='7D')),
 (Timestamp('2021-08-16 00:00:00', freq='7D'),
  Timestamp('2021-08-23 00:00:00', freq='7D'))]

## Задаем метрики и модели, по которым будем делать CV

In [9]:
# calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    'map@10': MAP(k=10),
    'novelty': MeanInvUserFreq(k=10),
}

# few simple models to compare
models = {
    'cosine_userknn': CosineRecommender(), # implicit 
    'tfidf_userknn': TFIDFRecommender(), 
}

# CV

- по двум моделям models
- по двум метрикам качества из metrics

In [10]:
%%time

results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.df.iloc[train_ids].copy()
    df_test = interactions.df.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()
    
    for model_name, model in models.items():
        userknn_model = UserKnn(model=model, N_users=50)
        userknn_model.fit(df_train)
    
        recos = userknn_model.predict(df_test)
    
        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )
    
        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)
        


==================== Fold 0
{'end': Timestamp('2021-07-12 00:00:00', freq='7D'),
 'i_split': 0,
 'start': Timestamp('2021-07-05 00:00:00', freq='7D'),
 'test': 204269,
 'test_items': 6199,
 'test_users': 83649,
 'train': 2939612,
 'train_items': 14397,
 'train_users': 603297}


  0%|          | 0/603297 [00:00<?, ?it/s]

  0%|          | 0/603297 [00:00<?, ?it/s]


==================== Fold 1
{'end': Timestamp('2021-07-19 00:00:00', freq='7D'),
 'i_split': 1,
 'start': Timestamp('2021-07-12 00:00:00', freq='7D'),
 'test': 216010,
 'test_items': 6314,
 'test_users': 84668,
 'train': 3239125,
 'train_items': 14730,
 'train_users': 646423}


  0%|          | 0/646423 [00:00<?, ?it/s]

...

  0%|          | 0/850489 [00:00<?, ?it/s]


==================== Fold 6
{'end': Timestamp('2021-08-23 00:00:00', freq='7D'),
 'i_split': 6,
 'start': Timestamp('2021-08-16 00:00:00', freq='7D'),
 'test': 298878,
 'test_items': 6679,
 'test_users': 110076,
 'train': 5051815,
 'train_items': 15577,
 'train_users': 906071}


  0%|          | 0/906071 [00:00<?, ?it/s]

  0%|          | 0/906071 [00:00<?, ?it/s]

CPU times: total: 1d 4h 28min 5s
Wall time: 1h 15min 17s


# Метрики качества по фолдам 

In [11]:
df_metrics = pd.DataFrame(results)
df_metrics

,fold,model,map@10,novelty
0,0,cosine_userknn,0.005572,7.100989
1,0,tfidf_userknn,0.009874,7.195373
2,1,cosine_userknn,0.004915,7.255019
3,1,tfidf_userknn,0.007821,7.369573
4,2,cosine_userknn,0.004343,7.389535
5,2,tfidf_userknn,0.007208,7.499991
6,3,cosine_userknn,0.004339,7.421055
7,3,tfidf_userknn,0.006852,7.539561
8,4,cosine_userknn,0.004246,7.476357
9,4,tfidf_userknn,0.006773,7.573736


## Metrics mean 


In [12]:
df_metrics.groupby('model').mean()[metrics.keys()]

,map@10,novelty
model,,
cosine_userknn,0.004412,7.398438
tfidf_userknn,0.007251,7.502899


# Домашнее задание: максимум 20 баллов 

## Основные пункты оценки
1. значение метрики на лидерборде
2. ревью кода в ноутбуке 
3. реализация сервиса для модели


Вы можете сделать НЕ ВСЕ пункты и все равно получить 20 баллов. Получение > 20 баллов будет расцениваться как 20.

## Подробности

### 1. Побейте метрику на лидерборде map@10 = 0.063 для userKnn модели с семинара (5 баллов)

### 2. Реализуйте эксперименты с кастомной моделю kNN с семинара. Результат - ноутбук(и) (максимум 15 баллов)

(Вы можете отрефакторить код из userknn.py по желанию или не трогать его) 

Что можно сделать в ноутбуке:

- придумать, что делать с холодными пользователями в тесте. Сделайте рекомендации для них (обратите внимание на rectools.models.popular) (3 балла)

- сделать кол-во рекомендаций равным N, а не меньше N (3 балла)

- реализовать тюнинг гиперпараметров (например, векторного расстояния или типов kNN моделей (implicit/rectools/...)) и сделать выводы (3 балла)

- реализовать другие варианты ранжированивания айтемов похожих пользователей и сделать выводы (3 балла)

- провести эксперименты с параметрами оффлайн валидации и сделать выводы (3 балла)

### 3. Оберните модель в сервис (максимум 12 баллов)

- предпочтительный онлайн вариант: обучаете модель в ноутбуке, сохраняете обученную модель (pickle, dill), при запуске сервиса ее поднимаете и запрашиваете рекомендации "на лету" (12 баллов)
- или оффлайн вариант: предварительно посчитайте рекомендации для всех пользователей, сохраните и запрашивайте их (6 баллов)

### Хороший pull request - это:

- наличие описания (в идеале что сделано - по пунктам)

- код по стандарту PEP8
- легкая читаемость и воспроизводимость кода
- комментарии и объяснения. В ipynb пользуйтесь силой маркдауна. В скриптах пишите комментарии и докстринг.
- обоснование схемы валидации
- анализ метрики качества
